In [1]:
#|default_exp net.test_net1

In [2]:
#|export
import fbdev

In [3]:
#|export
from typing import Type, Optional, Union, Any, Tuple, Dict
import asyncio

from fbdev.component import func_component, PortSpecCollection, PortTypeSpec, PortSpec, PortType
from fbdev.graph import Graph, NodeSpec, EdgeSpec
from fbdev.net import Net

In [4]:
#|export
@func_component()
def add_one(a:int) -> int:
    print("In add_one")
    return a+1

@func_component()
def copier(a:int) -> Tuple[int, int]:
    print("In copier")
    return a, a

@func_component()
def printer(a:int) -> None:
    print("In printer1:", a)
    
@func_component()
def sender(a:int):
    print("In sender")
    return a
    
g = Graph(PortSpecCollection(
    input=PortTypeSpec(in1=PortSpec(dtype=int)),
    output=PortTypeSpec(out=PortSpec(dtype=int))
))

g.add_node(NodeSpec(add_one))
g.add_node(NodeSpec(copier))
g.add_node(NodeSpec(printer))
g.add_node(NodeSpec(sender))

g.add_edge(EdgeSpec())
g.add_edge(EdgeSpec())
g.add_edge(EdgeSpec())
g.add_edge(EdgeSpec())
g.add_edge(EdgeSpec())

g.connect_edge_to_graph_port(PortType.INPUT, 'in1', 0)
g.connect_edge_to_node('add_one', PortType.INPUT, 'a', 0)

g.connect_edge_to_node('add_one', PortType.OUTPUT, 'out', 1)
g.connect_edge_to_node('copier', PortType.INPUT, 'a', 1)

g.connect_edge_to_node('copier', PortType.OUTPUT, 'out0', 2)
g.connect_edge_to_node('printer', PortType.INPUT, 'a', 2)

g.connect_edge_to_node('copier', PortType.OUTPUT, 'out1', 3)
g.connect_edge_to_node('sender', PortType.INPUT, 'a', 3)

g.connect_edge_to_node('sender', PortType.OUTPUT, 'out', 4)
g.connect_edge_to_graph_port(PortType.OUTPUT, 'out', 4)

In [5]:
g.display_mermaid()

```mermaid
graph TD
    add_one[add_one]
    copier[copier]
    printer[printer]
    sender[sender]
    0 -- input.in1 --> add_one
    add_one -- output.out --> copier
    copier -- output.out0 --> printer
    copier -- output.out1 --> sender
    sender -- output.out --> 0
```

In [6]:
#|export

async def run_net():
    output = await Net.async_execute_graph(g, 1)
    assert output['out'] is not None
    
def test_net1():
    asyncio.run(run_net())

In [7]:
await run_net()

In add_one
In copier
In printer1: 2
In sender
